In [13]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))


Ready to use Azure ML 1.31.0 to work with ws_Machine_Learning


In [20]:
from azureml.core import Experiment, Run
import pandas as pd

"""
Available metrics:
- Patients used for training
- Patients used for testing
- Batch dimensions
- Number of epochs
- Batch size
- Patch size
- Number of K folds
- Learning rates
- OACC
- ACC
- SEN
- SPE
- PRECISSION
- CONFUSION MATRIX
- Patients used to classify
- Predicted GT classification map
- Predicted and true GT classification maps
- Model loss and accuracy by epoch
- Time loading arguments
- Time preparing train data
- Time training CNN
- Time preparing test data
- Time predicting GT test image
- Time generating classification maps
"""

"""
Available experiments:
- exp-2-PyTorch-2D-CNN-train_noCV
- exp-2-PyTorch-2D-CNN-train_CV
- exp-3-PyTorch-2D-CNN-train_CV
- exp-3-PyTorch-2D-CNN-train_noCV
- exp-4-PyTorch-2D-CNN-train_CV
- exp-4-PyTorch-2D-CNN-train_noCV
"""

# Create Python list to iterate over all run experiments 
experiments = [#'exp-2-PyTorch-2D-CNN-train_noCV', 
                    'exp-2-PyTorch-2D-CNN-train_CV',
                    'exp-3-PyTorch-2D-CNN-train_CV',
                    'exp-3-PyTorch-2D-CNN-train_noCV',
                    'exp-4-PyTorch-2D-CNN-train_CV',
                    'exp-4-PyTorch-2D-CNN-train_noCV']


column_names = ['Experiment Name', 'Patients used for testing','OACC','ACC_tumor','SEN_tumor','SPE_tumor','Time loading arguments (s)',
                'Time preparing train data (s)', 'Time training CNN (s)', 'Time preparing test data (s)', 'Time predicting GT test image (s)',
                    'Time generating classification maps (s)']


for experiment_name in experiments:

    # Load experiment
    experiment = Experiment(ws, name=experiment_name)

    output_csv = './Results/Training_metrics/' + experiment_name + '.csv'

    # Initialize empty dataframe with column names
    df = pd.DataFrame(columns=column_names)
                                
    # Get all runs in the experiment and get their metrics (only if they are completed)
    for r in list(experiment.get_runs()):

        # Get the run by using the Workspace and the run id (only if the run is completed)
        if(r.status == 'Completed'):
            run = Run.get(ws, run_id=r.id)

            # Get all metrics from the run
            metrics = run.get_metrics()

            #print(metrics)

            df_temp = pd.DataFrame(data=[ [experiment_name, metrics['Patients used for testing'], metrics['OACC'], metrics['ACC'][1], metrics['SEN'][1],
                                        metrics['SPE'][1], metrics['Time loading arguments (s)'], metrics['Time preparing train data (s)'],
                                        metrics['Time training CNN (s)'], metrics['Time preparing test data (s)'],
                                        metrics['Time predicting GT test image (s)'], metrics['Time generating classification maps (s)']] ],
                                        columns=column_names)

            df = df.append(df_temp, ignore_index=True)

    df.to_csv(output_csv, index=False)